# How to run multiple agents on the same thread

In LangGraph API, a thread is not explicitly associated with a particular agent.
This means that you can run multiple agents on the same thread.
In this example, we will create two agents and then call them both on the same thread.

In [7]:
from langgraph_sdk import get_client

client = get_client()

openai_assistant = await client.assistants.create(graph_id="agent", config={"configurable": {"model_name": "openai"}})

# There should always be a default assistant with no configuration
assistants = await client.assistants.search()
default_assistant = [a for a in assistants if not a['config']][0]

We can see that these agents are different

In [8]:
openai_assistant

{'assistant_id': '13ecc353-a9a9-474b-a824-b6a343cd74b1',
 'graph_id': 'agent',
 'config': {'configurable': {'model_name': 'openai'}},
 'created_at': '2024-05-21T16:22:59.258447+00:00',
 'updated_at': '2024-05-21T16:22:59.258447+00:00',
 'metadata': {}}

In [9]:
default_assistant

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'created_at': '2024-05-18T00:19:39.688822+00:00',
 'updated_at': '2024-05-18T00:19:39.688822+00:00',
 'metadata': {'created_by': 'system'}}

We can now run it on the OpenAI assistant first.

In [14]:
thread = await client.threads.create()
input = {"messages": [{"role": "user", "content": "who made you?"}]}
async for event in client.runs.stream(thread['thread_id'], openai_assistant['assistant_id'], input=input, stream_mode='updates'):
    print(event)

StreamPart(event='metadata', data={'run_id': 'f90b3029-8669-4d70-976c-b70368e355d8'})
StreamPart(event='updates', data={'agent': {'messages': [{'content': 'I was created by OpenAI, a research organization focused on developing and advancing artificial intelligence technology.', 'additional_kwargs': {}, 'response_metadata': {'finish_reason': 'stop'}, 'type': 'ai', 'name': None, 'id': 'run-9801a5ba-2f3c-43de-89cf-c740debf36fc', 'example': False, 'tool_calls': [], 'invalid_tool_calls': []}]}})
StreamPart(event='end', data=None)


Now, we can run it on a different Anthropic-based assistant.

In [15]:
input = {"messages": [{"role": "user", "content": "and you?"}]}
async for event in client.runs.stream(thread['thread_id'], default_assistant['assistant_id'], input=input, stream_mode='updates'):
    print(event)

StreamPart(event='metadata', data={'run_id': 'c3521302-48ae-4c29-a0f2-5eb865cbc6d7'})
StreamPart(event='updates', data={'agent': {'messages': [{'content': "I am an AI assistant created by Anthropic to be helpful, harmless, and honest. I don't actually have a physical form or visual representation - I exist as a language model trained to have natural conversations.", 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'ai', 'name': None, 'id': 'run-4d05ffd7-0505-43e1-a068-0207c56b7665', 'example': False, 'tool_calls': [], 'invalid_tool_calls': []}]}})
StreamPart(event='end', data=None)
